In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# **1. Data Discovery**


In [2]:
# Data Loading
df = pd.read_csv("../data/car_prices_data.csv")

In [3]:
#Dataset view top rows
df.head()

,car_name,driven_km,fuel_type,transmission,ownership,manufacture,engine_cc,seats,km_per_cc,cc_per_seat,price_dollar
0,Hyundai Verna 1.4 VTVT,90228.0,Petrol,Manual,1st Owner,2014,2982.0,5,30.332250,596.0,5670.0
1,Hyundai Verna CRDi,100004.0,Diesel,Manual,2nd Owner,2009,1248.0,5,80.102661,249.0,3250.0
2,Honda City VX CVT,6144.0,Petrol,Automatic,1st Owner,2021,1497.0,5,3.867903,299.0,25960.0
3,Maruti Ciaz 1.4 Alpha,43016.0,Petrol,Manual,1st Owner,2017,1197.0,5,36.374141,239.0,10400.0
4,Mahindra TUV 300 T8 AMT,42032.0,Diesel,Automatic,2nd Owner,2016,2143.0,5,19.252918,428.0,7920.0


In [4]:
# Checking information about the dataset and number of rows and columns
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 5638 entries, 0 to 5637
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   car_name      5638 non-null   str    
 1   driven_km     5638 non-null   float64
 2   fuel_type     5638 non-null   str    
 3   transmission  5638 non-null   str    
 4   ownership     5638 non-null   str    
 5   manufacture   5638 non-null   int64  
 6   engine_cc     5623 non-null   float64
 7   seats         5638 non-null   int64  
 8   km_per_cc     5613 non-null   float64
 9   cc_per_seat   5623 non-null   float64
 10  price_dollar  5638 non-null   float64
dtypes: float64(5), int64(2), str(4)
memory usage: 484.6 KB


First of all, we split the data for later analysis

In [5]:
X = df.drop(columns=["price_dollar"])
targets = df["price_dollar"]
X_train, X_test, y_train, y_test = train_test_split(X, targets, test_size=0.2, random_state=42)
df = X_train.copy()
df["price_dollar"] = y_train


### Handle Duplicate Values

In [6]:
dup = df.duplicated().sum()
print(f'number of duplicated rows are {dup}')

number of duplicated rows are 70


In [7]:
df = df.drop_duplicates()

### Handle Missing Values/Null Values

In [8]:
df.isnull().sum()

car_name         0
driven_km        0
fuel_type        0
transmission     0
ownership        0
manufacture      0
engine_cc       11
seats            0
km_per_cc       20
cc_per_seat     11
price_dollar     0
dtype: int64

In [9]:
# Replace missing values with the median (robust to outliers)
df["engine_cc"] = df["engine_cc"].fillna(df["engine_cc"].median())

#### Minimize feature correlation

In [10]:
# Drop these 2 columns cause they are strongly correlated 
# with other present variable in the dataset engin_cc, seat, and driven_km
df = df.drop(columns=["km_per_cc", "cc_per_seat"])

### Handle null data

In [11]:
(df == 0).sum()

car_name        0
driven_km       0
fuel_type       0
transmission    0
ownership       0
manufacture     0
engine_cc       9
seats           0
price_dollar    2
dtype: int64

In [12]:
df[df["price_dollar"] == 0]

,car_name,driven_km,fuel_type,transmission,ownership,manufacture,engine_cc,seats,price_dollar
1960,Maruti Swift VDI,66000.0,Diesel,Manual,1st Owner,2014,2184.0,5,0.0
3457,Hyundai Verna VTVT 1.6 SX,19600.0,Petrol,Manual,1st Owner,2019,1591.0,5,0.0


In [13]:
df[df["engine_cc"] == 0]

,car_name,driven_km,fuel_type,transmission,ownership,manufacture,engine_cc,seats,price_dollar
428,Tata Harrier XM,33500.0,Diesel,Manual,1st Owner,2019,0.0,5,24220.0
1964,Ford Ecosport 1.5 DV5 MT Titanium,62500.0,Diesel,Manual,1st Owner,2015,0.0,5,6450.0
5273,Honda City E,77132.0,Petrol,Manual,1st Owner,2012,0.0,5,4830.0
2588,Hyundai Xcent 1.2 CRDi SX,42000.0,Diesel,Manual,1st Owner,2018,0.0,5,8310.0
1399,BMW X1 sDrive 20d xLine,22000.0,Diesel,Automatic,1st Owner,2020,0.0,5,43100.0
1198,Kia Seltos HTE D,35343.0,Diesel,Manual,1st Owner,2021,0.0,5,22780.0
1913,Honda CR-V 2.0L 2WD AT,74000.0,Petrol,Automatic,1st Owner,2016,0.0,7,29400.0
3735,Honda City i-VTEC CVT VX,65855.0,Petrol,Automatic,1st Owner,2019,0.0,5,15080.0
3283,Renault KWID Climber 1.0 AMT BSIV,30000.0,Petrol,Automatic,1st Owner,2019,0.0,5,6390.0


In [14]:
# Remove invalid price rows
df = df[df["price_dollar"] != 0]

# Fix invalid engine_cc values
median_cc = df.loc[df["engine_cc"] != 0, "engine_cc"].median()
df.loc[df["engine_cc"] == 0, "engine_cc"] = median_cc

The target variable (`price_dollar`) contained 2 missing values, which were removed, cant train without a target.  
For `engine_cc`, missing values were imputed using the median to preserve observations 
and ensure robustness to outliers.

### Understanding The Variables

In [15]:
df.columns

Index(['car_name', 'driven_km', 'fuel_type', 'transmission', 'ownership',
       'manufacture', 'engine_cc', 'seats', 'price_dollar'],
      dtype='str')

#### Numeric values sanity check

In [16]:
numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
df[numeric_columns].describe()

,driven_km,manufacture,engine_cc,seats,price_dollar
count,4438.000000,4438.000000,4438.000000,4438.000000,4438.000000
mean,63865.451780,2015.386435,1529.722623,5.250113,13840.286165
std,43043.460101,3.964352,577.665520,0.719724,13754.713832
min,250.000000,1995.000000,624.000000,2.000000,2500.000000
25%,33178.000000,2013.000000,1197.000000,5.000000,4880.000000
50%,60000.000000,2016.000000,1395.000000,5.000000,7220.000000
75%,85000.750000,2018.000000,1896.000000,5.000000,17675.000000
max,560000.000000,2022.000000,5950.000000,8.000000,69190.000000


#### Categorical values sanity check

In [17]:
for i in df.columns.tolist():
  print("No. of unique values in",i,"is",df[i].nunique())

No. of unique values in car_name is 1717
No. of unique values in driven_km is 2157
No. of unique values in fuel_type is 5
No. of unique values in transmission is 2
No. of unique values in ownership is 6
No. of unique values in manufacture is 26
No. of unique values in engine_cc is 134
No. of unique values in seats is 6
No. of unique values in price_dollar is 1988


#### Check Unique Values for each variable

In [18]:
df.groupby("fuel_type")["price_dollar"].median().sort_values(ascending=True)

fuel_type
Lpg          3280.0
Cng          5535.0
Petrol       6090.0
Diesel       9850.0
Electric    29370.0
Name: price_dollar, dtype: float64

In [19]:
df.groupby("transmission")["price_dollar"].median().sort_values(ascending=True)

transmission
Manual        5860.0
Automatic    31945.0
Name: price_dollar, dtype: float64

In [20]:
df.groupby("ownership")["price_dollar"].median().sort_values(ascending=True)

ownership
4th Owner     3630.0
5th Owner     4065.0
3rd Owner     4370.0
2nd Owner     6010.0
1st Owner     8290.0
0th Owner    40050.0
Name: price_dollar, dtype: float64

# **2. Data Wrangling**

### Minimize feature correlation and create new features

In [21]:
#Create vehicle age feature
CURRENT_YEAR = 2026
df["age"] = CURRENT_YEAR - df["manufacture"]

The vehicle age is computed from the current year and the manufacturing year.
This transformation reduces correlation and is more meaningful than using the raw year.

In [22]:
df = df.drop(columns=["manufacture"])

The original manufacture column is removed to avoid redundancy
after creating the age feature.

In [23]:
# Update numeric_columns
numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

#### Add new features extracted from car_names

A new feature `brand` is extracted from the `car_name` column
to capture brand-level information useful for price prediction.

In [24]:
#function to extract brand from car_name in file feature_engineering.py
from feature_engineering import extract_brand
df["brand"] = df["car_name"].apply(extract_brand)

In [25]:
# Verify that brand are created correctly
df[df["brand"] == "Other"].shape

(0, 10)

In [26]:
# car_name column is removed after extracting to avoid high-cardinality text features.
df.drop(columns=["car_name"], inplace=True)

In [27]:
# Categorical columns are identified for later encoding steps.
cat_columns = df.select_dtypes(exclude=['int64', 'float64']).columns.tolist()

Checkpoint

In [28]:
# Save cleaned data and column lists for later use in modeling
%store df
%store numeric_columns
%store cat_columns
%store X_test
%store y_test


Stored 'df' (DataFrame)
Stored 'numeric_columns' (list)
Stored 'cat_columns' (list)
Stored 'X_test' (DataFrame)
Stored 'y_test' (Series)
